In [87]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
    

In [88]:
#data cleaning

face_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')   #face cascade
eyes_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')                  #eye cascade



if face_detect.empty():
    raise IOError("Failed to load haarcascade_frontalface_default.xml")

if eyes_detect.empty():
    raise IOError("Failed to load haarcascade_eye.xml")


In [89]:
def cropped_image_with_2eyes(image_path):
    img=cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_detect.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        roi_gray=gray[y:y+h, x:x+w]
        roi_color=img[y:y+h,x:x+w]
        eyes=eyes_detect.detectMultiScale(roi_gray)
        if len(eyes)>=2:
            return roi_color
    return None

In [90]:
images_path='./images'
cropped_images='./cropped_images'


In [91]:
import os
img_dirs=[]
for entry in os.scandir(images_path):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [92]:
img_dirs

['./images\\elon_musk',
 './images\\jeff_bezos',
 './images\\pavel_durov',
 './images\\sam_altman',
 './images\\satya_nandella',
 './images\\sundar_pichai']

In [93]:
import shutil
if os.path.exists(cropped_images):
    shutil.rmtree(cropped_images)
os.makedirs(cropped_images)


In [ ]:
#commenting so that the code dont process the images again as we manually cleaned the images after the first run

"""cropped_images_dirs = []
name_dict = {}

for img_dir in img_dirs:
    name = img_dir.split('/')[-1]
    print(name)

    for entry in os.scandir(img_dir):
        roi_color = cropped_image_with_2eyes(entry.path)
        if roi_color is not None:
            # Create directory if not exists
            cropped_folder = os.path.join(cropped_images, name)
            os.makedirs(cropped_folder, exist_ok=True)

            # Create unique image path
            cropped_image_path = os.path.join(cropped_folder, entry.name)

            # Save image
            cv2.imwrite(cropped_image_path, roi_color)

            # Track image path and name
            cropped_images_dirs.append(cropped_image_path)
            name_dict[cropped_image_path] = name

    print(f'Processed {name} images')"""



images\elon_musk
Error: Unable to read image at ./images\elon_musk\04dtmd6g_elon-musk_625x300_22_November_24.jpg
Error: Unable to read image at ./images\elon_musk\1574363408-796.jpg
Error: Unable to read image at ./images\elon_musk\20250414030810_elonmuskssn.jpg
Error: Unable to read image at ./images\elon_musk\3VNRE7RXQDEWEFFS23ZEBGIZ2Q.jpg
Error: Unable to read image at ./images\elon_musk\679ae348eb49a-elon-musk-302611153-16x9.jpg
Error: Unable to read image at ./images\elon_musk\6EHE75VUFNNGJEM67MY6K3KBII.jpg
Error: Unable to read image at ./images\elon_musk\elon-musk-1-gty-gmh-250604_1749042286752_hpMain.jpg
Error: Unable to read image at ./images\elon_musk\elon-musk-1-rt-gmh-250528_1748436299357_hpMain.jpg
Error: Unable to read image at ./images\elon_musk\https%3A%2F%2Fcloudfront-us-east-2.images.arcpublishing.com%2Freuters%2FNPYZQCXOD5NHFDJXBOQLVZD7PM.jpg
Error: Unable to read image at ./images\elon_musk\spacex-founder-elon-musk-171701448-16x9_0.jpg
Error: Unable to read image at

In [117]:
#using wavelet transform to extract features
import pywt
def wavelet_transform(image_path,mode='haar',level=2):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    array=img
    #array=cv2.cvtColor(array, cv2.COLOR_BGR2GRAY)
    array=np.float32(array)
    array /= 255.0  # Normalize to [0, 1]
    coeffs = pywt.wavedec2(array, 'haar', level=level)
    coeffs_H=list(coeffs)
    coeffs_H[0] *=0  # Set the approximation coefficients to zero

    array_H = pywt.waverec2(coeffs_H, 'haar')
    array_H *= 255.0  # Scale back to [0, 255]
    array_H=np.uint8(array_H)  # Convert to uint8

    return array_H

In [121]:
class_dict = {}
count = 0
for namee in name_dict.values():
    if namee not in class_dict:
        namee= namee.split('\\')[-1]
        class_dict[namee] = count
        count += 1
class_dict

{'elon_musk': 63,
 'jeff_bezos': 125,
 'pavel_durov': 197,
 'sam_altman': 255,
 'satya_nandella': 281,
 'sundar_pichai': 330}

In [119]:
name_dict

{'./cropped_images\\images\\elon_musk\\-1x-1.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\0x0.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\1.elon_musk_spacex.jpg_0-sixteen_nine_0.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\108148546-17478234831747823476-39906072556-1080pnbcnews.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\121606874.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\1536x864_cmsv2_3bb817ce-a836-5de4-ba2a-35dedb6f1691-9151374.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\1749050680626.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\2025-03-05t203318z-423168903-rc277da2er36-rtrmadp-3-usa-trump-congress-musk.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\241219-elon-musk-vl-347p-f0e2a5.jpg': 'images\\elon_musk',
 './cropped_images\\images\\elon_musk\\250516-Musk-RS-cbcca6.jpg': 'images\\elon_musk',
 './cropped_images\\

In [123]:
X=[]
Y=[]

for training_image , namee in name_dict.items():
    namee= namee.split('\\')[-1]  # Extract the name from the path
    #print(f'Processing images for {namee}, path: {training_image}')
    
    img=cv2.imread(training_image)
    if img is None:
        continue
    scalled_raw_image = cv2.resize(img, (32,32))  # Resize to 32x32
    img_har= wavelet_transform(training_image)
    scalled_raw_har = cv2.resize(img_har, (32,32))
    combined_image = np.vstack((scalled_raw_image.reshape(32*32*3,1), scalled_raw_har.reshape(32*32,1)))
    X.append(combined_image)
    Y.append(class_dict[namee])


In [132]:
len(X), len(Y),X[0].shape

(301, 301, (4096, 1))

succesful addition

Training Time


In [148]:
# USE support vector machine to classify the images
from sklearn import svm
X = np.array(X).reshape(len(X), -1)  # Flatten the images
Y = np.array(Y)
model = svm.SVC(kernel='linear', C=1.0, random_state=42)
model.fit(X, Y)
#accuracy
model.score(X, Y)

1.0

In [152]:
#save the model
import joblib
model_path = 'face_recognition_model.pkl'
joblib.dump(model, model_path)
import json
# Save the class dictionary to a JSON file
class_dict_path = 'class_dict.json'
with open(class_dict_path, 'w') as f:
    json.dump(class_dict, f)


In [150]:
# Process each image, apply wavelet transform, and save the combined image
import numpy as np
def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None
    
    #cut the face and eyes then predict
    cropped_img = cropped_image_with_2eyes(image_path)
    if cropped_img is None:
        print(f"Error: No valid face with two eyes detected in {image_path}")
        return None
    img = cropped_img
    scalled_raw_image = cv2.resize(img, (32, 32))
    img_har = wavelet_transform(image_path)
    scalled_raw_har = cv2.resize(img_har, (32, 32))
    combined_image = np.vstack((scalled_raw_image.reshape(32*32*3, 1), scalled_raw_har.reshape(32*32, 1)))
    combined_image = combined_image.reshape(1, -1)  # Reshape for prediction
    prediction = model.predict(combined_image)
    return prediction[0]


In [151]:
path=str(input("Enter the path of the image to predict: "))
predicted_class = predict_image(path)
if predicted_class is not None:
    for name, class_id in class_dict.items():
        if class_id == predicted_class:
            print(f"The predicted class for the image is: {name}")
            break

The predicted class for the image is: jeff_bezos
